In [35]:
import sys
sys.path.append('../')

In [1]:
import geopandas as gpd

In [39]:
from enrich.enrich_CNRA import prepare_activity_table, prepare_project_table, capitalize_columns

In [31]:
gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\2023\CNRA_2023\CNRA_Tracker_Data_Export_20241120.gdb'

In [32]:
gpd.list_layers(gdb_path)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\core.py:130: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\core.py:130: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\core.py:130: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiPolygon' is converted to 'MultiPolygon Z'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))


,name,geometry_type
0,TREATMENT_POINT,Point
1,TREATMENT_LINE,MultiLineString
2,TREATMENT_POLY,MultiPolygon
3,PROJECT_POLY,MultiPolygon
4,Validation_Point_Errors,Point Z
5,Validation_Line_Errors,MultiLineString Z
6,Validation_Polygon_Errors,MultiPolygon Z
7,ACTIVITIES,None
8,Validation_Object_Errors,None


In [33]:
project_poly = gpd.read_file(gdb_path, driver='openFileGDB', layer='PROJECT_POLY')
treatment_poly = gpd.read_file(gdb_path, driver='openFileGDB', layer='TREATMENT_POLY')
activities = gpd.read_file(gdb_path, driver='openFileGDB', layer='ACTIVITIES')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [41]:
project_poly = capitalize_columns(project_poly.to_crs(3310))
treatment_poly = capitalize_columns(treatment_poly.to_crs(3310))
activities = capitalize_columns(activities)

In [43]:
input_features = treatment_poly.dropna(subset=['GlobalID_text']).copy()

input_features['PROJECTID_USER'] = input_features['PROJECTID_USER'].astype(str).str[:45] + '-CNRA'
input_features['TRMTID_USER'] = input_features['TRMTID_USER'].astype(str).str[:45] + '-CNRA'

# Part 2 Prepare Activity Table
activity_table = prepare_activity_table(activities)


merged_data = input_features.merge(
        activity_table,
        left_on='GlobalID_text',
        right_on='TREATMENTID_',
        how='inner'
    )

2025-03-21 09:32:23,833 INFO  [enrich.enrich_CNRA    ]     Part 2 Prepare Activity Table
2025-03-21 09:32:24,376 INFO  [enrich.enrich_CNRA    ]        step 2/17 remove milliseconds from dates
2025-03-21 09:32:24,392 INFO  [enrich.enrich_CNRA    ]        step 3/17 create standardized activity table
2025-03-21 09:32:24,396 INFO  [enrich.enrich_CNRA    ]        step 4/17 import activities into standardized table
2025-03-21 09:32:24,420 INFO  [enrich.enrich_CNRA    ]        step 5/17 calculate unique Treatment ID with postfix '-CNRA'


In [44]:
merged_data

,GlobalID,PROJECTID,TRMTID_USER_x,PROJECTID_USER,PROJECTNAME_,ORG_ADMIN_t,PRIMARY_OWNERSHIP_GROUP,PRIMARY_OBJECTIVE,SECONDARY_OBJECTIVE,TERTIARY_OBJECTIVE,...,Year,Year_txt,Act_Code,Crosswalk,Federal_FY,State_FY,TRMTID_USER_y,BATCHID_a,TRMT_GEOM,COUNTS_TO_MAS
0,{E8DAB1DA-9343-4970-88B5-2BFB9A5C89B5},{8614B280-DAC2-4E61-B757-BAC3159D81FC},50026-CNRA,40359-CNRA,San Diego River Bottom Fire Fuels Reduction,SDRC,LOCAL,OTHER_FUELS_REDUCTION,FUEL_BREAK,Other Fuels Reduction,...,<NA>,NaN,<NA>,NaN,<NA>,<NA>,50026-CNRA,NaN,POLYGON,YES
1,{6716C285-EF05-4DFB-93B9-68E78E6DBC50},{A2E12959-77AA-43AE-A254-8743BBCB92C3},50027-CNRA,40360-CNRA,Vegetation Management in and along the San Die...,SDRC,LOCAL,OTHER_FUELS_REDUCTION,FUEL_BREAK,Other Fuels Reduction,...,<NA>,NaN,<NA>,NaN,<NA>,<NA>,50027-CNRA,NaN,POLYGON,YES
2,{0D3B2617-AFF4-43C8-8681-D1DD6FDE90A6},{D78CC6DA-7534-49BE-902E-5B60D01E29EB},50030-CNRA,40365-CNRA,Alvarado Creek Bed Fire Fuel Reduction,SDRC,LOCAL,OTHER_FUELS_REDUCTION,Fuel Break,Other Fuels Reduction,...,<NA>,NaN,<NA>,NaN,<NA>,<NA>,50030-CNRA,NaN,POLYGON,YES
3,{124B9C9D-09C5-4138-A6DC-3F9E57493FB6},{D78CC6DA-7534-49BE-902E-5B60D01E29EB},50031-CNRA,40365-CNRA,Alvarado Creek Bed Fire Fuel Reduction,SDRC,LOCAL,OTHER_FUELS_REDUCTION,FUEL_BREAK,Other Fuels Reduction,...,<NA>,NaN,<NA>,NaN,<NA>,<NA>,50031-CNRA,NaN,POLYGON,YES
4,{1035CBEA-2BC3-4D37-AB29-F3BBD1C5FF2E},{5D2F679A-EAEE-44BA-A4BF-BD47360BFC2A},50032-CNRA,40363-CNRA,City of El Cajon Vegetation Management Program,SDRC,LOCAL,OTHER_FUELS_REDUCTION,OTHER_FUELS_REDUCTION,None,...,<NA>,NaN,<NA>,NaN,<NA>,<NA>,50032-CNRA,NaN,POLYGON,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27651,{467231D1-C091-42D2-85B0-B5250DE2DEEC},{E0C2E9C8-B4B5-4B62-81EA-5841DC0CCCA2},B-23-6619-12517 Fuel Reduction Lovers Leap-CNRA,B-23-6619-12517 Lovers Leap-CNRA,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,<NA>,NaN,<NA>,NaN,<NA>,<NA>,B-23-6619-12517 Fuel Reduction Lovers Leap-CNRA,NaN,POLYGON,NO
27652,{467231D1-C091-42D2-85B0-B5250DE2DEEC},{E0C2E9C8-B4B5-4B62-81EA-5841DC0CCCA2},B-23-6619-12517 Fuel Reduction Lovers Leap-CNRA,B-23-6619-12517 Lovers Leap-CNRA,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,<NA>,NaN,<NA>,NaN,<NA>,<NA>,B-23-6619-12517 Fuel Reduction Lovers Leap-CNRA,NaN,POLYGON,NO
27653,{5955017A-88D6-4ACF-AAA8-6DD36A24F7A1},{7F3049FA-D074-4BC8-B981-CD27A4FAA152},B-23-6619-12517 Fuel Reduction Twin Bridges-CNRA,B-23-6619-12517 Twin Bridges-CNRA,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,<NA>,NaN,<NA>,NaN,<NA>,<NA>,B-23-6619-12517 Fuel Reduction Twin Bridges-CNRA,NaN,POLYGON,YES
27654,{5955017A-88D6-4ACF-AAA8-6DD36A24F7A1},{7F3049FA-D074-4BC8-B981-CD27A4FAA152},B-23-6619-12517 Fuel Reduction Twin Bridges-CNRA,B-23-6619-12517 Twin Bridges-CNRA,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,<NA>,NaN,<NA>,NaN,<NA>,<NA>,B-23-6619-12517 Fuel Reduction Twin Bridges-CNRA,NaN,POLYGON,NO


In [55]:
merged_data = treatment_poly.merge(
        activity_table,
        left_on='GlobalID',
        right_on='TREATMENTID_',
        how='left'
    )

In [54]:
project_table_no_geom = project_poly.drop(columns=['geometry'])

In [57]:
cnra_flat = merged_data.merge(
        project_table_no_geom,
        left_on='GlobalID',
        right_on='GlobalID',
        how='left'
    )

In [58]:
cnra_flat

,GlobalID,PROJECTID,TRMTID_USER_x,PROJECTID_USER_x,PROJECTNAME_,ORG_ADMIN_t,PRIMARY_OWNERSHIP_GROUP,PRIMARY_OBJECTIVE,SECONDARY_OBJECTIVE,TERTIARY_OBJECTIVE,...,LATITUDE,LONGITUDE,GlobalID_text_y,CREATED_USER_y,CREATED_DATE_y,LAST_EDITED_USER_y,LAST_EDITED_DATE_y,VALIDATIONSTATUS_y,SHAPE_LENGTH_y,SHAPE_AREA_y
0,{1C9276FF-2734-452D-97F4-8B191EC6CF41},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50000,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
1,{2E17AAFC-8CDE-40CF-A7DD-6636EEE4C715},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50001,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
2,{78A1D43B-EC55-4556-A126-190FEACB4995},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50002,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
3,{DA9CFA34-53B0-450E-B8EC-64F181DDF244},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50003,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
4,{48387669-458F-4D90-900F-531E4929AA44},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50004,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27954,{467231D1-C091-42D2-85B0-B5250DE2DEEC},{E0C2E9C8-B4B5-4B62-81EA-5841DC0CCCA2},B-23-6619-12517 Fuel Reduction Lovers Leap,B-23-6619-12517 Lovers Leap,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
27955,{467231D1-C091-42D2-85B0-B5250DE2DEEC},{E0C2E9C8-B4B5-4B62-81EA-5841DC0CCCA2},B-23-6619-12517 Fuel Reduction Lovers Leap,B-23-6619-12517 Lovers Leap,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
27956,{5955017A-88D6-4ACF-AAA8-6DD36A24F7A1},{7F3049FA-D074-4BC8-B981-CD27A4FAA152},B-23-6619-12517 Fuel Reduction Twin Bridges,B-23-6619-12517 Twin Bridges,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
27957,{5955017A-88D6-4ACF-AAA8-6DD36A24F7A1},{7F3049FA-D074-4BC8-B981-CD27A4FAA152},B-23-6619-12517 Fuel Reduction Twin Bridges,B-23-6619-12517 Twin Bridges,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN


In [60]:
cnra_flat2

,GlobalID_x,PROJECTID,TRMTID_USER_x,PROJECTID_USER_x,PROJECTNAME_,ORG_ADMIN_t,PRIMARY_OWNERSHIP_GROUP,PRIMARY_OBJECTIVE,SECONDARY_OBJECTIVE,TERTIARY_OBJECTIVE,...,LATITUDE,LONGITUDE,GlobalID_text_y,CREATED_USER_y,CREATED_DATE_y,LAST_EDITED_USER_y,LAST_EDITED_DATE_y,VALIDATIONSTATUS_y,SHAPE_LENGTH_y,SHAPE_AREA_y
0,{1C9276FF-2734-452D-97F4-8B191EC6CF41},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50000,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,34.019479,-118.033500,{C2765772-95DB-434A-BB64-2BB7C7D83D89},reis.thomas@fire.ca.gov,2024-01-12 17:33:23+00:00,reis.thomas@fire.ca.gov,2024-03-14 22:15:27+00:00,2.0,5008.163082,873617.065506
1,{2E17AAFC-8CDE-40CF-A7DD-6636EEE4C715},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50001,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,34.019479,-118.033500,{C2765772-95DB-434A-BB64-2BB7C7D83D89},reis.thomas@fire.ca.gov,2024-01-12 17:33:23+00:00,reis.thomas@fire.ca.gov,2024-03-14 22:15:27+00:00,2.0,5008.163082,873617.065506
2,{78A1D43B-EC55-4556-A126-190FEACB4995},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50002,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,34.019479,-118.033500,{C2765772-95DB-434A-BB64-2BB7C7D83D89},reis.thomas@fire.ca.gov,2024-01-12 17:33:23+00:00,reis.thomas@fire.ca.gov,2024-03-14 22:15:27+00:00,2.0,5008.163082,873617.065506
3,{DA9CFA34-53B0-450E-B8EC-64F181DDF244},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50003,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,34.019479,-118.033500,{C2765772-95DB-434A-BB64-2BB7C7D83D89},reis.thomas@fire.ca.gov,2024-01-12 17:33:23+00:00,reis.thomas@fire.ca.gov,2024-03-14 22:15:27+00:00,2.0,5008.163082,873617.065506
4,{48387669-458F-4D90-900F-531E4929AA44},{C2765772-95DB-434A-BB64-2BB7C7D83D89},50004,40247,Rio Hondo College (RHC) Fuel Reduction Project,RMC,LOCAL,Other Fuels Reduction,Fire Prevention,None,...,34.019479,-118.033500,{C2765772-95DB-434A-BB64-2BB7C7D83D89},reis.thomas@fire.ca.gov,2024-01-12 17:33:23+00:00,reis.thomas@fire.ca.gov,2024-03-14 22:15:27+00:00,2.0,5008.163082,873617.065506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27954,{467231D1-C091-42D2-85B0-B5250DE2DEEC},{E0C2E9C8-B4B5-4B62-81EA-5841DC0CCCA2},B-23-6619-12517 Fuel Reduction Lovers Leap,B-23-6619-12517 Lovers Leap,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,38.799957,-120.139205,{E0C2E9C8-B4B5-4B62-81EA-5841DC0CCCA2},mami.odaya@fire.ca.gov,2024-11-20 17:45:54+00:00,mami.odaya@fire.ca.gov,2024-11-20 18:00:53+00:00,NaN,2313.772025,320266.263270
27955,{467231D1-C091-42D2-85B0-B5250DE2DEEC},{E0C2E9C8-B4B5-4B62-81EA-5841DC0CCCA2},B-23-6619-12517 Fuel Reduction Lovers Leap,B-23-6619-12517 Lovers Leap,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,38.799957,-120.139205,{E0C2E9C8-B4B5-4B62-81EA-5841DC0CCCA2},mami.odaya@fire.ca.gov,2024-11-20 17:45:54+00:00,mami.odaya@fire.ca.gov,2024-11-20 18:00:53+00:00,NaN,2313.772025,320266.263270
27956,{5955017A-88D6-4ACF-AAA8-6DD36A24F7A1},{7F3049FA-D074-4BC8-B981-CD27A4FAA152},B-23-6619-12517 Fuel Reduction Twin Bridges,B-23-6619-12517 Twin Bridges,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,38.811800,-120.124483,{7F3049FA-D074-4BC8-B981-CD27A4FAA152},mami.odaya@fire.ca.gov,2024-11-20 17:45:54+00:00,mami.odaya@fire.ca.gov,2024-11-20 18:00:53+00:00,NaN,1829.179766,206226.475188
27957,{5955017A-88D6-4ACF-AAA8-6DD36A24F7A1},{7F3049FA-D074-4BC8-B981-CD27A4FAA152},B-23-6619-12517 Fuel Reduction Twin Bridges,B-23-6619-12517 Twin Bridges,GGRF Kelsey Wildfire Resilience Project USFS,CCC,FEDERAL,OTHER_FUELS_REDUCTION,None,None,...,38.811800,-120.124483,{7F3049FA-D074-4BC8-B981-CD27A4FAA152},mami.odaya@fire.ca.gov,2024-11-20 17:45:

In [59]:
cnra_flat2 = merged_data.merge(
        project_table_no_geom,
        left_on='PROJECTID',
        right_on='GlobalID',
        how='left'
    )

In [62]:
project_poly.columns

Index(['GlobalID', 'PROJECTID_USER', 'AGENCY', 'ORG_ADMIN_p',
       'PROJECT_CONTACT', 'PROJECT_EMAIL', 'ADMINISTERING_ORG', 'PROJECT_NAME',
       'PROJECT_STATUS', 'PROJECT_START', 'PROJECT_END',
       'PRIMARY_FUNDING_SOURCE', 'PRIMARY_FUNDING_ORG', 'IMPLEMENTING_ORG',
       'LATITUDE', 'LONGITUDE', 'GlobalID_text', 'CREATED_USER',
       'CREATED_DATE', 'LAST_EDITED_USER', 'LAST_EDITED_DATE',
       'VALIDATIONSTATUS', 'SHAPE_LENGTH', 'SHAPE_AREA', 'geometry'],
      dtype='object')